## <a id='toc'>SIIM-ISIC Melanoma Classification using Light GBM</a>
## Identify melanoma in lesion images
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Table of Contents</h3>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#mel" role="tab" aria-controls="profile">What is Melanoma?<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#data" role="tab" aria-controls="messages">Studying the data<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action"  data-toggle="list" href="#eda" role="tab" aria-controls="settings">Exploratory Data Analysis<span class="badge badge-primary badge-pill">3</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#res" role="tab" aria-controls="settings">Resizing the Images<span class="badge badge-primary badge-pill">4</span></a> 
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#ext" role="tab" aria-controls="settings">Extract features from Images<span class="badge badge-primary badge-pill">5</span></a> 
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#sub" role="tab" aria-controls="settings">Train the model and predict the result<span class="badge badge-primary badge-pill">6</span></a>

<h3>References:</h3>
I would like to thank [Grandmaster Dieter](https://www.kaggle.com/christofhenkel) for sharing this awesome kernel to extract features from Images using a pretrained NN. Here's a link to his kernel:-
https://www.kaggle.com/christofhenkel/extract-image-features-from-pretrained-nn

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly_express as px
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from tqdm import tqdm, tqdm_notebook
import gc
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K
from lightgbm import LGBMClassifier

# <a id='mel'>1. What is Melanoma?</a>
<a href='#toc'><span class="label label-info">Go back to Table of Contents</span></a>
## -> Overview
<img src='https://www.mayoclinic.org/-/media/kcms/gbs/patient-consumer/images/2013/11/15/17/43/ds00190_-ds00439_im04411_mcdc7_melanomathu_jpg.jpg' style="width:500px;height:300px;">
Melanoma, the most serious type of skin cancer, develops in the cells (melanocytes) that produce melanin — the pigment that gives your skin its color. Melanoma can also form in your eyes and, rarely, inside your body, such as in your nose or throat.

The exact cause of all melanomas isn't clear, but exposure to ultraviolet (UV) radiation from sunlight or tanning lamps and beds increases your risk of developing melanoma. Limiting your exposure to UV radiation can help reduce your risk of melanoma.

The risk of melanoma seems to be increasing in people under 40, especially women. Knowing the warning signs of skin cancer can help ensure that cancerous changes are detected and treated before the cancer has spread. Melanoma can be treated successfully if it is detected early.
## -> Symptoms
Melanomas can develop anywhere on your body. They most often develop in areas that have had exposure to the sun, such as your back, legs, arms and face.

Melanomas can also occur in areas that don't receive much sun exposure, such as the soles of your feet, palms of your hands and fingernail beds. These hidden melanomas are more common in people with darker skin.

The first melanoma signs and symptoms often are:

A change in an existing mole
The development of a new pigmented or unusual-looking growth on your skin
Melanoma doesn't always begin as a mole. It can also occur on otherwise normal-appearing skin.
## -> Causes
<img src='https://www.mayoclinic.org/-/media/kcms/gbs/patient-consumer/images/2013/11/15/17/40/ds00190_-ds00439_-ds00924_-ds00925_im02400_c7_skincancerthu_jpg.jpg' style="width:500px;height:300px;">
Melanoma occurs when something goes wrong in the melanin-producing cells (melanocytes) that give color to your skin.

Normally, skin cells develop in a controlled and orderly way — healthy new cells push older cells toward your skin's surface, where they die and eventually fall off. But when some cells develop DNA damage, new cells may begin to grow out of control and can eventually form a mass of cancerous cells.

Just what damages DNA in skin cells and how this leads to melanoma isn't clear. It's likely that a combination of factors, including environmental and genetic factors, causes melanoma. Still, doctors believe exposure to ultraviolet (UV) radiation from the sun and from tanning lamps and beds is the leading cause of melanoma.

UV light doesn't cause all melanomas, especially those that occur in places on your body that don't receive exposure to sunlight. This indicates that other factors may contribute to your risk of melanoma.

# <a id='data'>2. Studying the data</a>
<a href='#toc'><span class="label label-info">Go back to Table of Contents</span></a>

In this section, I'll fetch the train and test data and study their distributions. Interestingly, this competition has both tabular as well as image data. So, I will be extracting the features from Images and then combine it with the tabular data and then make the predictions.

In [ ]:
df_train = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
df_test = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')

In [ ]:
display(df_train.info())
display(df_test.info())

In [ ]:
print('No of patients in train set(A): {}'.format(len(df_train.patient_id.unique().tolist())))
print('No of patients in test set(B): {}'.format(len(df_test.patient_id.unique().tolist())))
print('No of patients that are in train set but not test set(A-B): {}'.format(len(set(df_train.patient_id.unique()) - set(df_test.patient_id.unique()))))
print('No of patients that are in test set but not train set(B-A): {}'.format(len(set(df_test.patient_id.unique()) - set(df_train.patient_id.unique()))))
print('*'*64)
print('No of images in train set(C): {}'.format(len(df_train.image_name.unique().tolist())))
print('No of images in test set(D): {}'.format(len(df_test.image_name.unique().tolist())))
print('No of images that are in train set but not test set(C-D): {}'.format(len(set(df_train.image_name.unique()) - set(df_test.image_name.unique()))))
print('No of images that are in test set but not train set(D-C): {}'.format(len(set(df_test.image_name.unique()) - set(df_train.image_name.unique()))))

**Clearly there are no common patients and images between the train and test data. So, while training we should drop both these columns as these are not adding any value.**

Now, lets study the distribution of data across train and test set.

In [ ]:
PAPER_BGCOLOR = '#FFFFFF'
PLOT_BGCOLOR = '#FFFFFF'

In [ ]:
group_age_train = df_train.groupby('age_approx')['patient_id','image_name'].nunique()
group_age_test = df_test.groupby('age_approx')['patient_id','image_name'].nunique()
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.02,
                    subplot_titles=['Patients and their age',
                                   'Images available wrt age'])
fig.add_trace(go.Bar(y=group_age_train.index,x=group_age_train['patient_id'],orientation='h',name='Train',
                        legendgroup='Train',marker_color='rgb(119, 221, 119)',marker_line_color='black',
                        marker_line_width=1.5),1,1)
fig.add_trace(go.Bar(y=group_age_test.index,x=group_age_test['patient_id'],orientation='h',name='Test',
                       legendgroup='Test',marker_color='rgb(119, 158, 203)',marker_line_color='black',
                       marker_line_width=1.5),1,1)
fig.add_trace(go.Bar(y=group_age_train.index,x=group_age_train['image_name'],orientation='h',name='Train',
                     legendgroup='Train',marker_color='rgb(119, 221, 119)',marker_line_color='black',showlegend=False,
                    marker_line_width=1.5),1,2)
fig.add_trace(go.Bar(y=group_age_test.index,x=group_age_test['image_name'],orientation='h',name='Test',
                     legendgroup='Test',marker_color='rgb(119, 158, 203)',marker_line_color='black',showlegend=False,
                     marker_line_width=1.5),1,2)
fig.update_yaxes(title_text='Age',row=1,col=1,nticks=20,tickfont=dict(size=10),
                mirror=True,linecolor='black',linewidth=2)
fig.update_yaxes(mirror=True,linecolor='black',linewidth=2)
fig.update_xaxes(title_text='Number of Patients',row=1,col=1,mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_xaxes(title_text='Number of Images',row=1,col=2,mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_layout(width=700,paper_bgcolor=PAPER_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR, barmode='stack',hovermode='y unified',
                 margin=dict(l=0,r=0,t=0,b=0),legend=dict(title='<b>Dataset</b>',x=0.85,y=0.99))
fig.show()

In [ ]:
group_sex_train = df_train.groupby('sex')['patient_id','image_name'].nunique()
group_sex_test = df_test.groupby('sex')['patient_id','image_name'].nunique()
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.1,
                    subplot_titles=['Patients and their sex',
                                   'Images available wrt sex'])
fig.add_trace(go.Bar(y=group_sex_train.index,x=group_sex_train['patient_id'],orientation='h',name='Train',
                        legendgroup='Train',marker_color='rgb(119, 221, 119)',marker_line_color='black',
                        marker_line_width=2),1,1)
fig.add_trace(go.Bar(y=group_sex_test.index,x=group_sex_test['patient_id'],orientation='h',name='Test',
                       legendgroup='Test',marker_color='rgb(119, 158, 203)',marker_line_color='black',
                        marker_line_width=2),1,1)
fig.add_trace(go.Bar(y=group_sex_train.index,x=group_sex_train['image_name'],orientation='h',name='Train',
                 legendgroup='Train',marker_color='rgb(119, 221, 119)',showlegend=False,marker_line_color='black',
                        marker_line_width=2),1,2)
fig.add_trace(go.Bar(y=group_sex_test.index,x=group_sex_test['image_name'],orientation='h',name='Test',
                 legendgroup='Test',marker_color='rgb(119, 158, 203)',showlegend=False,marker_line_color='black',
                        marker_line_width=2),1,2)
fig.update_yaxes(title_text='Sex',row=1,col=1,nticks=20,tickfont=dict(size=10),
                mirror=True,linecolor='black',linewidth=2)
fig.update_yaxes(mirror=True,linecolor='black',linewidth=2)
fig.update_xaxes(title_text='Number of Patients',row=1,col=1,mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_xaxes(title_text='Number of Images',row=1,col=2,mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_layout(width=700,height=300,paper_bgcolor=PAPER_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR, barmode='stack',hovermode='y unified',
                 margin=dict(l=0,r=0,t=70,b=0),legend=dict(title='<b>Dataset</b>',x=0.35,y=1.25,orientation='h'))
fig.show()

In [ ]:
group_site_train = df_train.groupby('anatom_site_general_challenge')['patient_id','image_name'].nunique().sort_values(['patient_id','image_name'])
group_site_test = df_test.groupby('anatom_site_general_challenge')['patient_id','image_name'].nunique().sort_values(['patient_id','image_name'])
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.03,
                    subplot_titles=['Imaged site vs Patients',
                                   'Imaged site vs Images'])
fig.add_trace(go.Bar(y=group_site_train.index,x=group_site_train['patient_id'],orientation='h',name='Train',
                        legendgroup='Train',marker_color='rgb(119, 221, 119)',marker_line_color='black',
                        marker_line_width=2),1,1)
fig.add_trace(go.Bar(y=group_site_test.index,x=group_site_test['patient_id'],orientation='h',name='Test',
                       legendgroup='Test',marker_color='rgb(119, 158, 203)',marker_line_color='black',
                        marker_line_width=2),1,1)
fig.add_trace(go.Bar(y=group_site_train.index,x=group_site_train['image_name'],orientation='h',name='Train',
                 legendgroup='Train',marker_color='rgb(119, 221, 119)',showlegend=False,marker_line_color='black',
                        marker_line_width=2),1,2)
fig.add_trace(go.Bar(y=group_site_test.index,x=group_site_test['image_name'],orientation='h',name='Test',
                 legendgroup='Test',marker_color='rgb(119, 158, 203)',showlegend=False,marker_line_color='black',
                        marker_line_width=2),1,2)
fig.update_yaxes(title_text='Location of imaged site',row=1,col=1,nticks=20,tickfont=dict(size=10),
                mirror=True,linecolor='black',linewidth=2)
fig.update_yaxes(mirror=True,linecolor='black',linewidth=2)
fig.update_xaxes(title_text='Number of Patients',row=1,col=1,mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_xaxes(title_text='Number of Images',row=1,col=2,mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_layout(width=700,height=400,paper_bgcolor=PAPER_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR, barmode='stack',hovermode='y unified',
                 margin=dict(l=0,r=0,t=70,b=0),legend=dict(title='<b>Dataset</b>',x=0.35,y=1.17,orientation='h'))
fig.show()

In [ ]:
#Paths to train and test images
train_img_path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_img_path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'

In [ ]:
def get_image(img_name,is_train=True,transform=False):
    if is_train:
        path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
    else:
        path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
    if transform:
        img = load_image(path, img_name)
    else:
        path = os.path.join(path,img_name+'.jpg')
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
def show_data(df,rows,cols,is_train=True,transform=False):
    df.reset_index(inplace=True,drop=True)
    fig = plt.figure(figsize=(6,6),dpi=150)
    i = 1
    for r in range(rows):
        for c in range(cols):
            image_name = df.loc[i-1,'image_name']
            title = 'Patient id: '+df.loc[i-1,'patient_id']+'\n'\
              +'Site: '+str(df.loc[i-1,'anatom_site_general_challenge'])+'\n'\
              +'Sex: '+str(df.loc[i-1,'sex'])+'\n'\
              +'Approximate Age: '+str(int(df.loc[i-1,'age_approx']))
            ax = fig.add_subplot(rows,cols,i)
            img = get_image(image_name,is_train,transform)
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_title(title,fontsize=5)
            ax.imshow(img)
            i+=1
    return fig

**Let's take a look at some benign tumours from the train set.**

In [ ]:
fig = show_data(df_train[df_train['benign_malignant']=='benign'].iloc[:20,:],4,5)
fig.tight_layout()

**Now let's take a look at some malignant tumours from the train set.**

In [ ]:
fig = show_data(df_train[df_train['benign_malignant']=='malignant'].iloc[:20,:],4,5)
fig.tight_layout()

**Finally,let's check what we have to predict.**

In [ ]:
fig = show_data(df_test.iloc[:20,:],4,5,is_train=False)
fig.tight_layout()

# <a id='eda'>3. Exploratory Data Analysis</a>
<a href='#toc'><span class="label label-info">Go back to Table of Contents</span></a>

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/Pie_chart_of_incidence_and_malignancy_of_pigmented_skin_lesions.png/1920px-Pie_chart_of_incidence_and_malignancy_of_pigmented_skin_lesions.png' style="width:500px;height:300px;">
Various differential diagnoses of pigmented skin lesions, by relative rates upon biopsy and malignancy potential, including "melanoma" at right.

**Below plots show the analysis of Imaged sites with respect to Sex, Diagnosis and Tumour type.**

In [ ]:
fig = make_subplots(rows=1,cols=3,shared_yaxes=True, horizontal_spacing=0.03,
                    subplot_titles=['wrt Sex','wrt Diagnosis','wrt Tumour type'])
group = df_train.groupby(['anatom_site_general_challenge','sex'],as_index=False)['patient_id'].count().dropna()
for sex in group.sex.unique():
    group_n = group[group['sex']==sex].sort_values('patient_id')
    fig.append_trace(go.Bar(x=group_n['patient_id'],y=group_n['anatom_site_general_challenge'],orientation='h',
                            showlegend=False,name=sex,marker_line_color='black',marker_line_width=1.5),1,1)
group = df_train.groupby(['anatom_site_general_challenge','diagnosis'],as_index=False)['patient_id'].count().dropna()
for diag in group.diagnosis.unique():
    group_n = group[group['diagnosis']==diag].sort_values('patient_id')
    fig.append_trace(go.Bar(x=group_n['patient_id'],y=group_n['anatom_site_general_challenge'],orientation='h',
                            showlegend=False,name=diag,marker_line_color='black',marker_line_width=1.5),1,2)
group = df_train.groupby(['anatom_site_general_challenge','benign_malignant'],as_index=False)['patient_id'].count().dropna()
for type in group.benign_malignant.unique():
    group_n = group[group['benign_malignant']==type].sort_values('patient_id')
    fig.append_trace(go.Bar(x=group_n['patient_id'],y=group_n['anatom_site_general_challenge'],orientation='h',
                            showlegend=False,name=type,marker_line_color='black',marker_line_width=1.5),1,3)
fig.update_yaxes(title_text='Imaged site',row=1,col=1,nticks=20,tickfont=dict(size=10))
fig.update_yaxes(mirror=True,linecolor='black',linewidth=2)
fig.update_xaxes(row=1,col=1,mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_xaxes(title_text='Number of Patients',row=1,col=2,
                 mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_xaxes(row=1,col=3,mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_layout(width=700,height=400,paper_bgcolor=PAPER_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR, 
                  barmode='stack',hovermode='y unified',title='Imaged site Analysis',
                 margin=dict(l=0,r=0,t=70,b=0))
fig.show()

**Below plots show the analysis of Sex of patients with respect to their Diagnosis and Tumour type.**

In [ ]:
fig = make_subplots(rows=1,cols=2,shared_yaxes=True, horizontal_spacing=0.03,
                    subplot_titles=['Sex wrt Diagnosis','Sex wrt Tumour type'])
group = df_train.groupby(['sex','diagnosis'],as_index=False)['patient_id'].count().dropna()
for diag in group.diagnosis.unique():
    group_n = group[group['diagnosis']==diag].sort_values('patient_id')
    fig.append_trace(go.Bar(x=group_n['patient_id'],y=group_n['sex'],orientation='h',
                            showlegend=False,name=diag,marker_line_color='black',marker_line_width=1.5),1,1)
group = df_train.groupby(['sex','benign_malignant'],as_index=False)['patient_id'].count().dropna()
for type in group.benign_malignant.unique():
    group_n = group[group['benign_malignant']==type].sort_values('patient_id')
    fig.append_trace(go.Bar(x=group_n['patient_id'],y=group_n['sex'],orientation='h',
                            showlegend=False,name=type,marker_line_color='black',marker_line_width=1.5),1,2)
fig.update_yaxes(title_text='Sex',row=1,col=1)
fig.update_xaxes(title_text='Number of patients',row=1,col=1,
                 mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_xaxes(title_text='Number of patients',row=1,col=2,
                 mirror=True,linecolor='black',linewidth=2,gridcolor='darkgrey')
fig.update_yaxes(mirror=True,linecolor='black',linewidth=2)
fig.update_layout(width=700,height=300,paper_bgcolor=PAPER_BGCOLOR,plot_bgcolor=PLOT_BGCOLOR, 
                  barmode='stack',hovermode='y unified',title='Sex Analysis',
                 margin=dict(l=0,r=0,t=70,b=0))
fig.show()

In [ ]:
del group
del group_age_test
del group_age_train
del group_n
del group_sex_test
del group_sex_train
del group_site_test
del group_site_train
gc.collect();

# <a id='res'>4. Resizing the Images</a>
<a href='#toc'><span class="label label-info">Go back to Table of Contents</span></a>

In this section I'll be defining the function to resize the images so that it becomes easier to extract features from them as we have a common ground. Also, I'll be plotting some random resized images from the train set and test set.

**The Image size that I have chosen is 256X256.**

In [ ]:
#Size to resize(256,256,3)
img_size = 256

#Paths to train and test images
train_img_path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_img_path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'

def resize_image(img):
    old_size = img.shape[:2]
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    img = cv2.resize(img, (new_size[1],new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0,0,0]
    new_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return new_img

def load_image(path, img_id):
    path = os.path.join(path,img_id+'.jpg')
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    new_img = resize_image(img)
    new_img = preprocess_input(new_img)
    return new_img

<h3>Here's a look at some of the resized images of Benign Tumours.</h3>

In [ ]:
fig = show_data(df_train[df_train['benign_malignant']=='benign'].iloc[:20,:],4,5,transform=True)
fig.tight_layout()

<h3>Also, lets take a look at some of the resized images of Malignant Tumours.</h3>

In [ ]:
fig = show_data(df_train[df_train['benign_malignant']=='malignant'].iloc[:20,:],4,5,transform=True)
fig.tight_layout()

<h3>Finally, lets also take a look at some images from the test set.</h3>

In [ ]:
fig = show_data(df_test.iloc[:20,:],4,5,is_train=False,transform=True)
fig.tight_layout()

Clearly, Resizing the images has lead to deteoration of their quality but still features seems to be intact. I could also train resize them 512X512 to extract more features.

# <a id='ext'>5. Extract features from Images</a>
<a href='#toc'><span class="label label-info">Go back to Table of Contents</span></a>

<div class="alert alert-info" role="alert">
 This entire process of extraction of features from Images using DenseNet121 takes more than 2 hours. So, to save on time I have created a public dataset: https://www.kaggle.com/anshuls235/melanomaextractedfeatures. This conatins all the extracted features and can be used by anyone. I'm also using the same. Also, as a result I have commented the code in this section, but can be used by anyone for reference to extract any number of features as per their convenience.
</div>

Here, I'll be extracting Image features using a pretrained Neural network. Later, using these features and the tabular data I'll train a LGBM classifier and make the predictions. The pretrained NN used here is a DenseNet121. Per image I'll be extracting 256 features for both the train and test set.
<img src='https://imgur.com/wWHWbQt.jpg' style="width:800px;height:200px;">
**DenseNet Architecture**
<img src='https://imgur.com/oiTdqJL.jpg' style="width:500px;height:300px;">
Normally DenseNet121 would output 1024 features after GlobalAveragePooling. To further narrow it down, I again pool 4 features each.

In [ ]:
# img_size = 256
# batch_size = 16 #16 images per batch

# train_img_ids = df_train.image_name.values
# n_batches = len(train_img_ids)//batch_size + 1

# #Model to extract image features
# inp = Input((256,256,3))
# backbone = DenseNet121(input_tensor=inp, include_top=False)
# x = backbone.output
# x = GlobalAveragePooling2D()(x)
# x = Lambda(lambda x: K.expand_dims(x,axis=-1))(x)
# x = AveragePooling1D(4)(x)
# out = Lambda(lambda x: x[:,:,0])(x)

# m = Model(inp,out)

In [ ]:
# features = {}
# for b in tqdm_notebook(range(n_batches)):
#     start = b*batch_size
#     end = (b+1)*batch_size
#     batch_ids = train_img_ids[start:end]
#     batch_images = np.zeros((len(batch_ids),img_size,img_size,3))
#     for i,img_id in enumerate(batch_ids):
#         try:
#             batch_images[i] = load_image(train_img_path,img_id)
#         except:
#             pass
#     batch_preds = m.predict(batch_images)
#     for i,img_id in enumerate(batch_ids):
#         features[img_id] = batch_preds[i]

In [ ]:
# train_feats = pd.DataFrame.from_dict(features, orient='index')
# #Save for future reference 
# train_feats.to_csv('train_img_features.csv')
# train_feats.head()

In [ ]:
# test_img_ids = df_test.image_name.values
# n_batches = len(test_img_ids)//batch_size + 1

In [ ]:
# features = {}
# for b in tqdm_notebook(range(n_batches)):
#     start = b*batch_size
#     end = (b+1)*batch_size
#     batch_ids = test_img_ids[start:end]
#     batch_images = np.zeros((len(batch_ids),img_size,img_size,3))
#     for i,img_id in enumerate(batch_ids):
#         try:
#             batch_images[i] = load_image(test_img_path,img_id)
#         except:
#             pass
#     batch_preds = m.predict(batch_images)
#     for i,img_id in enumerate(batch_ids):
#         features[img_id] = batch_preds[i]

In [ ]:
# test_feats = pd.DataFrame.from_dict(features, orient='index')
# test_feats.to_csv('test_img_features.csv')
# test_feats.head()

# <a id='sub'>6. Train the model and predict the results</a>
<a href='#toc'><span class="label label-info">Go back to Table of Contents</span></a>

In this final section, I'll be combining the extracted image features and tabular data available into a single dataframe. I'll be using LGBM Classifier to classify the test data into benign/malignant.

In [ ]:
train_feats = pd.read_csv('/kaggle/input/siimisic-extracted-features-from-256x256-image/train_img_features (1).csv')
test_feats = pd.read_csv('/kaggle/input/siimisic-extracted-features-from-256x256-image/test_img_features (1).csv')
train_feats.set_index(train_feats.columns[0],inplace=True)
test_feats.set_index(test_feats.columns[0],inplace=True)

In [ ]:
#Combine the image and tabular data
df_train_full = pd.merge(df_train, train_feats, how='inner', left_on='image_name', right_index=True)
df_test_full = pd.merge(df_test, test_feats, how='inner', left_on='image_name', right_index=True)

#Drop the unwanted columns
train = df_train_full.drop(['image_name','patient_id','diagnosis','benign_malignant'],axis=1)
test = df_test_full.drop(['image_name','patient_id'],axis=1)

#Label Encode categorical features
train.sex.fillna('NaN',inplace=True)
test.sex.fillna('NaN',inplace=True)
train.anatom_site_general_challenge.fillna('NaN',inplace=True)
test.anatom_site_general_challenge.fillna('NaN',inplace=True)
le_sex = LabelEncoder()
le_site = LabelEncoder()
train.sex = le_sex.fit_transform(train.sex)
test.sex = le_sex.transform(test.sex)
train.anatom_site_general_challenge = le_site.fit_transform(train.anatom_site_general_challenge)
test.anatom_site_general_challenge = le_site.transform(test.anatom_site_general_challenge)

Since, the data is imbalanced meaning there are a lot of records for benign tumours while very less records for malign tumour. So, I'll be using a Stratified K-fold for validation.

In [ ]:
folds = StratifiedKFold(n_splits= 10, shuffle=True)
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])
feature_importance_df = pd.DataFrame()
features = [f for f in train.columns if f != 'target']
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[features], train['target'])):
    train_X, train_y = train[features].iloc[train_idx], train['target'].iloc[train_idx]
    valid_X, valid_y = train[features].iloc[valid_idx], train['target'].iloc[valid_idx]
    clf = LGBMClassifier(
        #device='gpu',
        n_estimators=1000,
        learning_rate=0.001,
        max_depth=8,
        colsample_bytree=0.5,
        num_leaves=50,
        random_state=0
    )
    print('*****Fold: {}*****'.format(n_fold))
    clf.fit(train_X, train_y, eval_set=[(train_X, train_y), (valid_X, valid_y)], 
            eval_metric= 'auc', verbose= 20, categorical_feature=[0,2],early_stopping_rounds= 20)

    oof_preds[valid_idx] = clf.predict_proba(valid_X, num_iteration=clf.best_iteration_)[:, 1]
    sub_preds += clf.predict_proba(test[features], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importances_
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
    del clf, train_X, train_y, valid_X, valid_y
    gc.collect()

Here's a graph showcasing the feature importances averaged over folds.

In [ ]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (averaged over folds)')
    plt.tight_layout()
    plt.savefig('feature_importance.jpg')
    
display_importances(feature_importance_df)

Make the submission!

In [ ]:
submission = pd.DataFrame({
    "image_name": df_test.image_name, 
    "target": sub_preds
})
submission.to_csv('submission.csv', index=False)

<img src='https://media.giphy.com/media/lD76yTC5zxZPG/giphy.gif' style="width:500px;height:300px;">

<a href="#toc" class="btn btn-success btn-lg active" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Go to TOP</a>
# Do leave an upvote if you liked my work :)